In [66]:
import zipfile
import numpy as np
import os
from skimage.transform import resize

In [ ]:
def unpack(filename):
    with zipfile.ZipFile(filename) as (zf):
        zf.extractall()

In [22]:
MAX_IMG_VALUE = 255

In [2]:
unpack('all.zip')
unpack('test.zip')
unpack('training.zip')

Now we have files:
* IdLookupTable.csv
* training.csv
* test.csv


In [8]:
with open('IdLookupTable.csv', 'r') as IdLookupTable_csv:
    print(IdLookupTable_csv.readline())
    print(IdLookupTable_csv.readline())
    print(IdLookupTable_csv.readline())
    print(IdLookupTable_csv.readline())

RowId,ImageId,FeatureName,Location

1,1,left_eye_center_x

2,1,left_eye_center_y

3,1,right_eye_center_x



In [26]:
with open('training.csv', 'r') as training:
    print(training.readline())
    print(np.array([int(num)/MAX_IMG_VALUE for num in training.readline().split(',')[-1].split(' ')]).reshape(96,96))
    print(np.array([float(pt) for pt in training.readline().split(',')[:-1]]))
    print(np.array([float(pt) for pt in training.readline().split(',')[:-1]]).shape)

left_eye_center_x,left_eye_center_y,right_eye_center_x,right_eye_center_y,left_eye_inner_corner_x,left_eye_inner_corner_y,left_eye_outer_corner_x,left_eye_outer_corner_y,right_eye_inner_corner_x,right_eye_inner_corner_y,right_eye_outer_corner_x,right_eye_outer_corner_y,left_eyebrow_inner_end_x,left_eyebrow_inner_end_y,left_eyebrow_outer_end_x,left_eyebrow_outer_end_y,right_eyebrow_inner_end_x,right_eyebrow_inner_end_y,right_eyebrow_outer_end_x,right_eyebrow_outer_end_y,nose_tip_x,nose_tip_y,mouth_left_corner_x,mouth_left_corner_y,mouth_right_corner_x,mouth_right_corner_y,mouth_center_top_lip_x,mouth_center_top_lip_y,mouth_center_bottom_lip_x,mouth_center_bottom_lip_y,Image

[[0.93333333 0.9254902  0.92941176 ... 0.98039216 0.98039216 0.98039216]
 [0.92156863 0.93333333 0.9254902  ... 0.97647059 0.98039216 0.98431373]
 [0.92941176 0.9254902  0.92941176 ... 0.98431373 0.98431373 0.98039216]
 ...
 [0.72941176 0.71764706 0.70980392 ... 0.20392157 0.22352941 0.23529412]
 [0.74117647 0.73725

In [95]:
def parse_imgs_and_keypoints(file, file_path = '', resize_h = 100, resize_w = 100, max_imgs = None):
    with open(os.path.join(file_path,file), 'r') as data:
        #remove header
        data.readline()
        
        images = np.empty((0,96,96,1), float)
        key_points = np.empty([0,30], float)
        
        num_imgs = 0
        for line in data:
            if len(line.split(',')) <31:
                continue
            row = line.split(',')
            
   
            row_pts = np.array([float(pt) if pt != '' else None for pt in row[:-1]])
  
            
            image = np.array([float(num)/MAX_IMG_VALUE for num in row[-1].split(' ')])

            image = image.reshape(1,96,96,1)

            #try:
            #    images = np.append(images, image, axis = 0)
            #except:
            #    print(image.shape)
            images = np.vstack((images,image))
            key_points = np.vstack((key_points, row_pts))
            
            num_imgs += 1
            if max_imgs is not None and num_imgs >= max_imgs:
                break
    return key_points, images

In [96]:
key_pts, imgs = parse_imgs_and_keypoints('training.csv', max_imgs = 100)

In [98]:
imgs.max()

1.0

In [65]:
key_pts

array([[66.03356391, 39.00227368, 30.22700752, ..., 72.93545865,
        43.13070677, 84.48577444],
       [64.33293617, 34.9700766 , 29.9492766 , ..., 70.26655319,
        45.46791489, 85.48017021],
       [65.05705263, 34.90964211, 30.90378947, ..., 70.19178947,
        47.27494737, 78.65936842],
       ...,
       [67.57628571, 31.16768571, 30.41914286, ..., 66.02657143,
        53.40257143, 81.34885714],
       [71.11064151, 35.92664151, 28.7474717 , ..., 71.81403774,
        46.55637736, 88.27411321],
       [67.67177143, 34.95579429, 30.89828571, ..., 71.1168    ,
        48.36548571, 80.9232    ]])

In [ ]:
with open('training.csv', 'r') as training:
    